In [2]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
stop = set(stopwords.words('english'))
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import string
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
import csv
from csv import reader
tit = []
with open("/content/COVID-19_title.csv", 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        tit.append(row)

In [5]:
tit[:10]

[['clinical features culture-proven mycoplasma pneumoniae infections king abdulaziz university hospital jeddah saudi arabia'],
 ['nitric oxide pro-inflammatory mediator lung diseas'],
 ['surfactant protein-d pulmonary host defens'],
 ['role endothelin-1 lung diseas'],
 ['gene expression epithelial cells response pneumovirus infect'],
 ['sequence requirements strand transfer nidovirus discontinuous subgenomic synthesi'],
 ['debate transfusing normal haemoglobin levels improve outcom'],
 ['21st international symposium intensive care emergency medicine brussels belgium 20-23 march 2001'],
 ['heme oxygenase-1 carbon monoxide pulmonary medicin'],
 ['technical description rods real-time public health surveillance system']]

In [6]:
val = []
for i in tit:
  val.append(i[0])

In [7]:
val[:10]
print(len(val))

242108


Splitting the dataset into 2 equal halves

In [8]:
from sklearn.model_selection import train_test_split
data1,data2 = train_test_split(val, train_size=0.5,random_state=42)
print(data1[:5])
print(data2[:5])

['respiratory health allergies farm environment design methods enrollment observational wisconsin infant study cohort wisc research propos', 'inefficient infection soluble receptor-resistant mutants murine coronavirus cells expressing mhvr2 receptor', 'keeping high quality health care greek inflammatory bowel disease patients sars-cov-2', 'covid-19 doctors sound alarm hospital transmiss', 'clinical course molecular viral shedding among asymptomatic symptomatic patients with sars-cov-2 infection community treatment center republic korea']
['generation defective avian coronavirus infectious bronchitis virus defective coronavirus', 'infectious diseases civets back china menu', 'clinical implications covid-19 pandemic dental educ', 'rapid decay anti-sars-cov-2 antibodies persons mild covid-19', 'evidence respiratory viruses interactions asymptomatic preschool-aged children']


**Running LDA individually on the data1**

In [9]:
#Using bag of words

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = set(stopwords.words('english'))
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
doc_set = data1
# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    texts.append(tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
corpus[:2]


[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1)],
 [(16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 2),
  (25, 1),
  (26, 1)]]

In [10]:
preview = corpus[-1]
for i in range(len(preview)):
    print("Word {} (\"{}\") appears {} time.".format(preview[i][0], 
                                               dictionary[preview[i][0]], preview[i][1]))

Word 39 ("19") appears 1 time.
Word 41 ("covid") appears 1 time.
Word 102 ("approach") appears 1 time.
Word 536 ("crisi") appears 1 time.
Word 1220 ("medicine") appears 1 time.
Word 1388 ("reply") appears 1 time.
Word 1478 ("editor") appears 1 time.
Word 1484 ("letter") appears 1 time.
Word 1523 ("oral") appears 1 time.
Word 11786 ("tele") appears 1 time.


In [11]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.34430804348396765),
 (1, 0.1788545663154484),
 (2, 0.19131486738407796),
 (3, 0.3400944808128881),
 (4, 0.2341381408508976),
 (5, 0.2677494554844608),
 (6, 0.11004607247967105),
 (7, 0.25021942252001556),
 (8, 0.2071485775801503),
 (9, 0.19920455010658578),
 (10, 0.2825185534184156),
 (11, 0.156979908656268),
 (12, 0.10862698341672329),
 (13, 0.14042564029774737),
 (14, 0.4186978501983728),
 (15, 0.31951144124583264)]


In [12]:
lda_model = gensim.models.LdaMulticore(corpus, num_topics=8, id2word=dictionary, passes=2, workers=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.014*"laparoscopic" + 0.012*"single" + 0.009*"surgery" + 0.009*"treatment" + 0.008*"tri" + 0.007*"randomized" + 0.007*"controlled" + 0.006*"using" + 0.006*"versus" + 0.006*"studi"
Topic: 1 
Words: 0.023*"virus" + 0.019*"human" + 0.019*"infectious" + 0.013*"influenza" + 0.012*"respiratory" + 0.010*"cell" + 0.010*"diseas" + 0.008*"emerging" + 0.007*"diseases" + 0.006*"viruses"
Topic: 2 
Words: 0.037*"coronavirus" + 0.029*"disease" + 0.026*"patients" + 0.024*"2019" + 0.023*"respiratory" + 0.022*"covid" + 0.022*"19" + 0.021*"acute" + 0.017*"severe" + 0.015*"clinical"
Topic: 3 
Words: 0.049*"covid" + 0.048*"19" + 0.008*"cross" + 0.006*"sectional" + 0.005*"review" + 0.005*"letter" + 0.005*"editor" + 0.005*"management" + 0.005*"current" + 0.004*"patient"
Topic: 4 
Words: 0.016*"based" + 0.012*"using" + 0.010*"detection" + 0.009*"model" + 0.007*"system" + 0.006*"review" + 0.006*"time" + 0.006*"evaluation" + 0.005*"network" + 0.005*"real"
Topic: 5 
Words: 0.118*"covid" + 0.115

In [13]:
lda_model = gensim.models.LdaMulticore(corpus, num_topics=7, id2word=dictionary, passes=40, workers=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.081*"covid" + 0.078*"19" + 0.020*"pandemic" + 0.018*"health" + 0.018*"pandem" + 0.014*"care" + 0.010*"review" + 0.009*"during" + 0.007*"impact" + 0.007*"systematic"
Topic: 1 
Words: 0.011*"single" + 0.011*"studi" + 0.010*"laparoscopic" + 0.006*"treatment" + 0.006*"study" + 0.006*"surgery" + 0.006*"tri" + 0.006*"patients" + 0.006*"term" + 0.005*"randomized"
Topic: 2 
Words: 0.016*"infectious" + 0.012*"emerging" + 0.010*"diseas" + 0.009*"diseases" + 0.006*"using" + 0.006*"system" + 0.005*"immunity" + 0.005*"based" + 0.005*"multiple" + 0.004*"deep"
Topic: 3 
Words: 0.053*"respiratory" + 0.022*"syndrome" + 0.022*"acute" + 0.019*"detection" + 0.015*"infections" + 0.014*"severe" + 0.013*"viral" + 0.013*"coronavirus" + 0.008*"virus" + 0.008*"middle"
Topic: 4 
Words: 0.064*"covid" + 0.064*"19" + 0.035*"coronavirus" + 0.033*"disease" + 0.028*"patients" + 0.022*"2019" + 0.013*"clinical" + 0.012*"case" + 0.010*"pneumonia" + 0.010*"novel"
Topic: 5 
Words: 0.051*"sars" + 0.045*"c

In [14]:
pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=ccc2d2b014fa6a40ece0f69dc1fc333bc428c6c8ea1a4d57fac357924609d672
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [15]:
import pyLDAvis
import pyLDAvis.gensim 

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.259579 -0.054306       1        1  23.867206
4     -0.163081  0.115549       2        1  17.379041
5      0.201203 -0.052194       3        1  16.554723
1      0.046121  0.314634       4        1  12.251054
6     -0.189364 -0.163174       5        1  12.221887
3      0.176076 -0.009084       6        1   9.824823
2      0.188624 -0.151425       7        1   7.901266, topic_info=          Term          Freq         Total Category  logprob  loglift
41       covid  44664.000000  44664.000000  Default  30.0000  30.0000
39          19  43601.000000  43601.000000  Default  29.0000  29.0000
38        sars   9801.000000   9801.000000  Default  28.0000  28.0000
30         cov   8700.000000   8700.000000  Default  27.0000  27.0000
27           2   8437.000000   8437.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
79       human    315.148535   3213.705907   Topic7  -5.6784   0.2160
102   approach    259.665140   1394.132303   Topic7  -5.8720   0.8575
202      model    240.063855   2169.664231   Topic7  -5.9505   0.3367
2024    feline    201.882357    536.948454   Topic7  -6.1237   1.5599
489     common    194.223979    426.210974   Topic7  -6.1624   1.7522

[396 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
893       3  0.866427        1
893       4  0.096730        1
893       6  0.035928        1
39        1  0.501011       19
39        2  0.295309       19
...     ...       ...      ...
59        4  0.096212     with
709       1  0.998675     work
986       1  0.999022  workers
1632      7  0.995253        x
8448      5  0.994466        å

[533 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 6, 2, 7, 4, 3])

In [16]:
lda_model_8_topics = gensim.models.LdaMulticore(corpus, num_topics=8, id2word=dictionary, passes=2, workers=2)

for idx, topic in lda_model_8_topics.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

Topic: 0 
Words: 0.085*"covid" + 0.084*"19" + 0.027*"coronavirus" + 0.021*"disease" + 0.020*"2019" + 0.019*"patients" + 0.010*"clinical" + 0.009*"novel" + 0.006*"infection" + 0.006*"pneumonia"
Topic: 1 
Words: 0.023*"virus" + 0.019*"coronavirus" + 0.019*"infectious" + 0.014*"human" + 0.009*"protein" + 0.008*"diseas" + 0.007*"emerging" + 0.007*"cell" + 0.006*"viral" + 0.006*"diseases"
Topic: 2 
Words: 0.038*"review" + 0.018*"systematic" + 0.013*"analysi" + 0.011*"meta" + 0.007*"patients" + 0.007*"term" + 0.006*"case" + 0.006*"studi" + 0.006*"based" + 0.005*"long"
Topic: 3 
Words: 0.005*"epidemic" + 0.005*"porcine" + 0.005*"pressure" + 0.004*"diarrhea" + 0.004*"ventilation" + 0.003*"virus" + 0.003*"blood" + 0.003*"infection" + 0.003*"stress" + 0.003*"canine"
Topic: 4 
Words: 0.024*"virus" + 0.010*"single" + 0.009*"laparoscopic" + 0.009*"using" + 0.006*"protein" + 0.006*"based" + 0.005*"vaccine" + 0.005*"cell" + 0.005*"hepatitis" + 0.004*"model"
Topic: 5 
Words: 0.054*"respiratory" + 0.03

In [17]:
import pyLDAvis
import pyLDAvis.gensim 

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_8_topics, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.301162 -0.015274       1        1  24.977004
0     -0.242645 -0.070521       2        1  19.096748
4      0.171487  0.071905       3        1  10.776823
6      0.133981 -0.065293       4        1  10.477763
5      0.054956 -0.139323       5        1   9.563935
1      0.131631 -0.173589       6        1   9.541458
2     -0.018702  0.251385       7        1   9.239938
3      0.070455  0.140710       8        1   6.326331, topic_info=           Term          Freq         Total Category  logprob  loglift
41        covid  44085.000000  44085.000000  Default  30.0000  30.0000
39           19  43056.000000  43056.000000  Default  29.0000  29.0000
38         sars   9963.000000   9963.000000  Default  28.0000  28.0000
30          cov   8831.000000   8831.000000  Default  27.0000  27.0000
27            2   8500.000000   8500.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
68        virus    249.243726   7042.844314   Topic8  -5.6907  -0.5809
20    infection    226.743133   4721.783104   Topic8  -5.7853  -0.2757
64       effect    164.721570   1435.682177   Topic8  -6.1049   0.5953
1258       pati    169.087975   1880.360623   Topic8  -6.0787   0.3517
802        mask    139.611261    356.391381   Topic8  -6.2702   1.8233

[621 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
8604      6  0.984676        18
39        1  0.564334        19
39        2  0.434085        19
39        4  0.000046        19
39        5  0.000232        19
...     ...       ...       ...
2077      5  0.065928  zoonotic
2077      6  0.917491  zoonotic
8448      4  0.988688         å
8451      4  0.986666         æ
8453      4  0.980250         è

[1743 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 5, 7, 6, 2, 3, 4])

**Running LDA individually on data2**

In [18]:
#Using bag of words

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = set(stopwords.words('english'))
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
doc_set = data2
# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    texts.append(tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
corpus[:2]


[[(0, 1), (1, 1), (2, 2), (3, 2), (4, 1), (5, 1), (6, 1)],
 [(5, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)]]

In [19]:
preview = corpus[-1]
for i in range(len(preview)):
    print("Word {} (\"{}\") appears {} time.".format(preview[i][0], 
                                               dictionary[preview[i][0]], preview[i][1]))

Word 12 ("19") appears 1 time.
Word 14 ("covid") appears 1 time.
Word 51 ("effect") appears 1 time.
Word 462 ("influenza") appears 1 time.
Word 2009 ("fighting") appears 1 time.
Word 3816 ("alert") appears 1 time.
Word 5469 ("seasonal") appears 1 time.
Word 56248 ("superposed") appears 1 time.


In [20]:
lda_model = gensim.models.LdaMulticore(corpus, num_topics=8, id2word=dictionary, passes=1, workers=3, random_state=42)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

Topic: 0 
Words: 0.014*"laparoscopic" + 0.010*"single" + 0.009*"surgery" + 0.008*"using" + 0.008*"treatment" + 0.007*"randomized" + 0.007*"term" + 0.007*"versus" + 0.007*"tri" + 0.006*"controlled"
Topic: 1 
Words: 0.101*"covid" + 0.099*"19" + 0.020*"pandemic" + 0.017*"health" + 0.016*"pandem" + 0.012*"care" + 0.007*"during" + 0.007*"impact" + 0.006*"outbreak" + 0.005*"medical"
Topic: 2 
Words: 0.030*"respiratory" + 0.014*"influenza" + 0.012*"infections" + 0.010*"infection" + 0.009*"children" + 0.008*"syndrome" + 0.007*"middle" + 0.007*"viral" + 0.007*"among" + 0.006*"east"
Topic: 3 
Words: 0.062*"covid" + 0.062*"19" + 0.042*"coronavirus" + 0.032*"patients" + 0.030*"disease" + 0.023*"2019" + 0.020*"acute" + 0.017*"severe" + 0.016*"clinical" + 0.012*"respiratory"
Topic: 4 
Words: 0.018*"virus" + 0.017*"protein" + 0.013*"cell" + 0.009*"human" + 0.009*"coronavirus" + 0.008*"cells" + 0.008*"model" + 0.007*"host" + 0.006*"using" + 0.006*"based"
Topic: 5 
Words: 0.090*"sars" + 0.079*"cov" + 0

In [21]:
import pyLDAvis
import pyLDAvis.gensim 

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.263373  0.026163       1        1  27.194112
3     -0.218263  0.043216       2        1  16.562255
4      0.183756  0.084002       3        1  11.548748
6      0.139069  0.146351       4        1   9.944816
2     -0.084855  0.070792       5        1   9.843417
5      0.168813  0.133471       6        1   9.733502
0     -0.079149 -0.178537       7        1   9.710171
7      0.154003 -0.325458       8        1   5.462979, topic_info=             Term          Freq         Total Category  logprob  loglift
14          covid  44333.000000  44333.000000  Default  30.0000  30.0000
12             19  43494.000000  43494.000000  Default  29.0000  29.0000
27           sars  10319.000000  10319.000000  Default  28.0000  28.0000
22            cov   9024.000000   9024.000000  Default  27.0000  27.0000
19              2   8735.000000   8735.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
1456  angiotensin    291.127249    721.263340   Topic8  -5.3929   1.9999
4563            t    163.770185    269.065670   Topic8  -5.9682   2.4107
604        states    163.401132    407.018229   Topic8  -5.9705   1.9945
1210        liver    165.685979    763.282884   Topic8  -5.9566   1.3796
51         effect    169.595412   1368.053416   Topic8  -5.9333   0.8194

[455 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
209       3  0.905256        1
209       5  0.093988        1
12        1  0.724690       19
12        2  0.275299       19
19        6  0.999893        2
...     ...       ...      ...
82        2  0.820963     with
82        7  0.047790     with
385       1  0.998676     work
706       1  0.999227  workers
961       4  0.993348     zika

[647 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 5, 7, 3, 6, 1, 8])

In [22]:
#trying for number of topics = 5 as seen from visualisation
lda_model_new = gensim.models.LdaMulticore(corpus, num_topics=5, id2word=dictionary, passes=40, workers=3, random_state=42)

for idx, topic in lda_model_new.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))


Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

Topic: 0 
Words: 0.010*"laparoscopic" + 0.008*"using" + 0.007*"single" + 0.007*"surgery" + 0.005*"versus" + 0.005*"term" + 0.005*"long" + 0.005*"treatment" + 0.004*"cancer" + 0.004*"model"
Topic: 1 
Words: 0.110*"covid" + 0.107*"19" + 0.016*"pandemic" + 0.013*"health" + 0.012*"pandem" + 0.010*"care" + 0.006*"during" + 0.006*"impact" + 0.006*"outbreak" + 0.005*"patients"
Topic: 2 
Words: 0.054*"sars" + 0.047*"cov" + 0.045*"2" + 0.015*"respiratory" + 0.011*"detection" + 0.010*"infection" + 0.010*"systematic" + 0.010*"review" + 0.009*"influenza" + 0.008*"infections"
Topic: 3 
Words: 0.045*"coronavirus" + 0.035*"disease" + 0.030*"patients" + 0.023*"2019" + 0.021*"acute" + 0.016*"severe" + 0.016*"clinical" + 0.015*"respiratory" + 0.012*"syndrome" + 0.011*"novel"
Topic: 4 
Words: 0.025*"virus" + 0.011*"protein" + 0.011*"coronavirus" + 0.010*"human" + 0.009*"cell" + 0.006*"viral" + 0.005*"vaccine" + 0.005*"cells" + 0.005*"potential" + 0.005*"porcine"


In [23]:
import pyLDAvis
import pyLDAvis.gensim 

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_new, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.231509 -0.199612       1        1  34.537209
4      0.282276  0.011623       2        1  17.801801
3     -0.068488  0.136112       3        1  17.532228
2      0.108635 -0.177674       4        1  16.356257
0     -0.090913  0.229551       5        1  13.772505, topic_info=            Term          Freq         Total Category  logprob  loglift
14         covid  44352.000000  44352.000000  Default  30.0000  30.0000
12            19  43480.000000  43480.000000  Default  29.0000  29.0000
27          sars  10261.000000  10261.000000  Default  28.0000  28.0000
22           cov   8953.000000   8953.000000  Default  27.0000  27.0000
19             2   8670.000000   8670.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
306     approach    492.896202   1375.861845   Topic5  -5.7910   0.9560
278        study    519.949663   2392.896044   Topic5  -5.7376   0.4560
1404  evaluation    444.210057   1383.002728   Topic5  -5.8950   0.8468
443         case    476.167960   2905.027201   Topic5  -5.8256   0.1741
448     learning    397.974799    793.130540   Topic5  -6.0050   1.2929

[286 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
209       2  0.940795        1
209       4  0.058257        1
12        1  0.999986       19
19        4  0.999920        2
1507      4  0.998297     2009
...     ...       ...      ...
917       2  0.997757    vitro
1712      1  0.999219     what
82        1  0.095723     with
82        3  0.904113     with
706       1  0.999140  workers

[375 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 4, 3, 1])